In [ ]:
import pymysql#db
import math
import sys
import pandas as pd#표
from future.utils import iteritems
from collections import Counter#갯수
import os
import csv#csv
from konlpy.tag import Okt#konlpy 라이브러리
twitter = Okt()

##db
db = pymysql.connect(host='localhost', user='root', password='****' ,db='documents', charset='utf8')
# prepare a cursor object using cursor() method


cursor = db.cursor()

# execute SQL query using execute() method.

cursor.execute("SELECT VERSION()")
# Fetch a single row using fetchone() method.

data = cursor.fetchone()

#db연결확인
print("Database version : %s " % data)

#csv형식의 뉴스 테이블(culture) 읽기, 읽은 후 documents 리스트에 기사 본문 저장
documents=[]
try:
    with db.cursor() as cursor:
        sql = "select * from culture"
        cursor.execute("set names utf8")
        cursor.execute(sql)
        result = cursor.fetchall()
        
        for i in result:
            documents.append(i[2])
finally:
    db.close()
    
print(len(documents))

#tf,tf-idf 기록 비교를 위한 명사 갯수 상위 6개 추출 함수
def get_tf(csv, lank=6):
    spliter = Okt()
    nouns = spliter.nouns(csv)
    count = Counter(nouns)
    
    return_dict = {}  
    for n, c in count.most_common(lank):#most_common(n):n개의 가장 많이 나오는 요소와 카운트의 리스트를 반환
        return_dict[n]=c
        #temp = {'tag': n, 'count': c}
        #return_tf.append(temp)    
    return return_dict


##idf구하는 함수
def get_idf(data1,data2,len):
    return_dict = {}
    
    score=get_score(data1,data2)#score dict 형태
    
    #idf=1
    for n in data1:
        #idf=math.log10((len-score.get(n))/score.get(n)) 
        idf=math.log10(len/score.get(n)) 
        return_dict[n]=idf
 
    return return_dict


#단어가 포함된 문서의수
def get_score(data1,data2):
    score_list={}
   
    for i in data1: 
        a=0    
        for n in range(len(data2)):
                    if i in data2[n]:
                       a+=1
        score_list[i]=a#자기 자신 제외하기
    return score_list


#명사 갯수 상위 6개 리스트
tf=[]

#idf 리스트 dict형태로 저장
idf=[]

#tf-idf (단어: 값) 리스트
tf_idf_list=[]

#카운트제외 명사만 저장
nouns_data=[]

#카운트 저장
nouns_dict=[]
a=0
#for i in line:
for i in documents:
    tag=get_tf(i)
    #tag=get_tf(i[1])#line 테스트시
    tf.append(tag)
    
    #모든 기사의 명사
    spliter = Okt()
    #nouns = spliter.nouns(i[1])#line 테스트시
    nouns = spliter.nouns(i)
    
    count = Counter(nouns)
    nouns_data.append(list(count.keys()))
    nouns_dict.append(count)
   
     # if not tag:
    #          continue             
      #  except:
      #      pass
def main(argv):
    
    #print("IDF값:",get_idf(list(tf[0].keys()),nouns_data,len(tf)))
    for i in nouns_data:
        idf.append(get_idf(i,nouns_data,len(nouns_data)))
        
    #print("TF",nouns_dict[1])
    #print("IDF",idf)

    for data in nouns_data:
        #print(len(data))
        tf_idf={}
        for i in range(len(data)):
            #print(nouns_dict[nouns_data.index(data)])
            #print(data[i]) 
            tf_idf[data[i]]=idf[nouns_data.index(data)].get(data[i])*nouns_dict[nouns_data.index(data)].get(data[i])
        tf_idf_list.append(tf_idf)
    print("tf-idf완료")                
if __name__ == '__main__':
    main(sys.argv)

In [ ]:
##db 컬럼 추가 후 태그 저장
db = pymysql.connect(host='localhost', user='root', password='rlaqh190' ,db='documents', charset='utf8')
# prepare a cursor object using cursor() method
cursor = db.cursor()

#컬럼 추가하기
#cursor.execute("ALTER TABLE culture ADD tags1 VARCHAR(100) NOT NULL")
#cursor.execute("ALTER TABLE culture ADD tags2 VARCHAR(100) NOT NULL") 
#cursor.execute("ALTER TABLE culture ADD tags3 VARCHAR(100) NOT NULL") 
#cursor.execute("ALTER TABLE culture ADD tags4 VARCHAR(100) NOT NULL") 
#cursor.execute("ALTER TABLE culture ADD tags5 VARCHAR(100) NOT NULL") 
#cursor.execute("ALTER TABLE culture ADD tags6 VARCHAR(100) NOT NULL")  
#cursor.execute("ALTER TABLE culture ADD tags6 VARCHAR(100) NOT NULL") 
#cursor.execute("ALTER TABLE culture ADD taglist CHAR(100) NOT NULL")

try:
    with db.cursor() as cursor:
        for i in range(len(tf_idf_list)):
            save=[]
            if not tf_idf_list[i]:
                     pass
            else:
                save.extend(sorted(tf_idf_list[i], reverse=True, key=tf_idf_list[i].get)[:6])
                
                #기사에 맞게 태그 집어넣기 (리스트와 sql id 비교)
                sql= "UPDATE `culture` SET taglist='%s'WHERE id= '%d'" % (save[0]+","+save[1]+","+save[2]+","+save[3]+","+save[4]+","+save[5],i+1)
                cursor.execute(sql)
                #print(save[0],save[1],save[2],save[3],save[4],save[5])                  
    db.commit()
    print("완료")
                       
finally:
    db.close              